# Spike for LangChain RAG

## Install libary
```shell
cd ../backend
poetry add langchain
poetry add langchain_community
poetry add langchain_chroma
poetry add langchain-openai
```

In [2]:
## setup langchain openai client

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

[2024-09-09 11:03:41 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 11:03:41 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 11:03:42 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 11:03:42 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 11:03:42 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 11:03:42 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 11:03:42 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 11:03:42 - httpx:146 - 

In [3]:
## preview

import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is Task Decomposition?")

[2024-09-09 11:03:45 - langchain_community.utils.user_agent:11 - WARNING] USER_AGENT environment variable not set, consider setting it to identify your requests.
[2024-09-09 11:03:46 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 11:03:46 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 11:03:46 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 11:03:46 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/certifi/cacert.pem'
[2024-09-09 11:03:46 - httpx:80 - DEBUG] load_ssl_context verify=True cert=None trust_env=True http2=False
[2024-09-09 11:03:46 - httpx:146 - DEBUG] load_verify_locations cafile='/Users/now/Documents/llm-code/children_ai_assistant/.venv/lib/python3.12/site-packages/

'Task Decomposition is the process of breaking down complex tasks into smaller, manageable steps to simplify problem-solving. It involves using techniques like Chain of Thought (CoT) to guide models in thinking step-by-step and generating multiple reasoning possibilities through structures like Tree of Thoughts. This approach aids in planning and enhances model performance by making intricate tasks easier to tackle.'

In [4]:
prompt

ChatPromptTemplate(input_variables=['context', 'question'], metadata={'lc_hub_owner': 'rlm', 'lc_hub_repo': 'rag-prompt', 'lc_hub_commit_hash': '50442af133e61576e74536c6556cefe1fac147cad032f4377b60c436e6cdcb6e'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:"))])

In [5]:
docs

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [6]:
len(docs)

1

In [7]:
splits

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refi